In [1]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import pickle as pkl

# load tokenized dataset

In [2]:
dataset_dir_path = '../../data/processed/tokenized_data/'
with open(dataset_dir_path + 'train_data.pkl', 'rb') as f:
    tokenized_train_data = pkl.load(f)

with open(dataset_dir_path + 'valid_data.pkl', 'rb') as f:
    tokenized_valid_data = pkl.load(f)

In [3]:
print(f'Input(de) {tokenized_train_data[0][0]}')
print(f'Output(en) {tokenized_train_data[0][1]}')

Input(de) tensor([   2,   21,   85,  256,   31,   86,   22,   93,    7,   16,  114, 5645,
        3245,    3])
Output(en) tensor([   2,   19,   25,   15, 1197,  817,   17,   58,   84,  332, 1319,    3])


# load vocab

In [4]:
vocab_dir_path = '../../data/processed/vocab/'

with open(vocab_dir_path + 'token2idx_de.pkl', 'rb') as f:
    token2idx_de= pkl.load(f)
with open(vocab_dir_path + 'token2idx_en.pkl', 'rb') as f:
    token2idx_en = pkl.load(f)
with open(vocab_dir_path + 'idx2token_de.pkl', 'rb') as f:
    idx2token_de = pkl.load(f)
with open(vocab_dir_path + 'idx2token_en.pkl', 'rb') as f:
    idx2token_en = pkl.load(f)

# making the batch

In [5]:
batch_size = 128
PAD_INDEX = token2idx_de['<pad>']
START_INDEX = token2idx_en['<start>']
END_INDEX = token2idx_en['<end>']

In [6]:
def generate_batch(data_batch):
    batch_src = []
    batch_tgt = []
    for src, tgt in data_batch:
        batch_src.append(src)
        batch_tgt.append(tgt)
    
    batch_src = pad_sequence(batch_src, padding_value=PAD_INDEX)
    batch_tgt = pad_sequence(batch_tgt, padding_value=PAD_INDEX)

    return batch_src, batch_tgt

In [7]:
train_iter = DataLoader(tokenized_train_data, batch_size=batch_size, shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(tokenized_valid_data, batch_size=batch_size, shuffle=True, collate_fn=generate_batch)

In [8]:
# show train_iter
# each column is a text
list(train_iter)[0]

(tensor([[  2,   2,   2,  ...,   2,   2,   2],
         [  5,   5,   5,  ...,  84,  21,   5],
         [ 12, 525,  12,  ...,  47, 282,  12],
         ...,
         [  1,   1,   1,  ...,   1,   1,   1],
         [  1,   1,   1,  ...,   1,   1,   1],
         [  1,   1,   1,  ...,   1,   1,   1]]),
 tensor([[  2,   2,   2,  ...,   2,   2,   2],
         [  6,   6,   6,  ...,   6,  19,   6],
         [ 12, 693,  12,  ..., 159, 131,  12],
         ...,
         [  1,   1,   1,  ...,   1,   1,   1],
         [  1,   1,   1,  ...,   1,   1,   1],
         [  1,   1,   1,  ...,   1,   1,   1]]))

In [13]:
for src, tgt in train_iter:
    print(src)
    print(tgt)
    print(f'src shape : {src.shape}')
    print(f'tgt.shape : {tgt.shape}')
    break

tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [  14,  129,    5,  ...,   21,   21,    5],
        [  38,  115, 6863,  ...,  115,   31, 1565],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])
tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [   6,  146,    6,  ...,   19,   19,    6],
        [  39,   25, 1921,  ...,   25,   36,  204],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])
src shape : torch.Size([30, 128])
tgt.shape : torch.Size([28, 128])


# Model

In [9]:
import math
import torch
import torch.nn as nn
from torch import Tensor

## token embedding

In [10]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, embedding_size):
        super().__init__()
        # nn.Embedding is a simple lookup table.
        # if token index is set, it will return the corresponding embedding vector.
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.embedding_size = embedding_size
    
    def forward(self, tokens: Tensor):
        # the reason for this multiply is to align the range of the values
        # It is to make the positional encoding relatively smaller. 
        # This means the original meaning in the embedding vector won’t be lost 
        # when we add them together.
        return self.embedding(tokens.long()) * math.sqrt(self.embedding_size)

## positional encoging

In [11]:
class PositionalEncoding(nn.Module):
    
    def __init__(self, embedding_size: int, dropout: float, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        
        den = torch.exp(-torch.arange(0, embedding_size, 2) * math.log(10000) / embedding_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        embedding_pos = torch.zeros((maxlen, embedding_size))
        embedding_pos[:, 0::2] = torch.sin(pos * den) # extract even element (start:stop:step)
        embedding_pos[:, 1::2] = torch.cos(pos * den) # extract odd element (start:stop:step)
        embedding_pos = embedding_pos.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('embedding_pos', embedding_pos) # positional encoding is not updated by learning

    def forward(self, token_embedding: Tensor):
        print(f'token_embedding : {token_embedding.shape}')
        print(f'positional_encoding : {self.embedding_pos.shape}') # self.embedding_pos is defined by self.register_fuffer.
        # self.embedding_pos is not updated by learning
        print(f'positional_encoding : {self.embedding_pos[:token_embedding.size(0), :].shape}')
        return self.dropout(token_embedding + self.embedding_pos[:token_embedding.size(0), :])

## masking

In [25]:
def generate_square_subsequent_mask(seq_len, PAD_INDEX):
    # seq_len is the length of the sentence
    # The mask is used to prevent the model from looking ahead in the sequence.
    # The mask is a square matrix of size (seq_len, seq_len)
    # The upper triangle of the matrix is filled with -inf
    # The lower triangle of the matrix is filled with 0
    mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).T
    mask = mask.float().masked_fill(mask == 1, float('-inf')).masked_fill(mask == 0, float(0.0)) # 1 -> -inf, 0 -> 0
    return mask

def create_mask(src, tgt, PAD_INDEX):
    seq_len_src = src.shape[0] # word number in one sentence in source
    seq_len_tgt = tgt.shape[0] # word number in one sentence in target

    mask_tgt = generate_square_subsequent_mask(seq_len_tgt)
    mask_src = torch.zeros((seq_len_src, seq_len_src), device=device).type(torch.bool)

    padding_mask_src = (src == PAD_INDEX).transpose(0, 1)
    padding_mask_tgt = (tgt == PAD_INDEX).transpose(0, 1)

    return mask_src, mask_tgt, padding_mask_src, padding_mask_tgt

In [28]:
seq_len_src = src.shape[0]
mask_src = torch.zeros((seq_len_src, seq_len_src)).type(torch.bool)

In [29]:
mask_src

tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, Fals